<a href="https://colab.research.google.com/github/GeeksGhost/Deep_learning_challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [11]:
# Identifying variables that serves as dependent variables and features of independent variables
# Dependent variables
y = application_df['IS_SUCCESSFUL'].values

# independent variables
X = application_df.drop(columns= ['EIN', 'NAME', 'IS_SUCCESSFUL']).values

application_df = application_df.drop(columns= ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [12]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [13]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_types = application_df['APPLICATION_TYPE'].value_counts()

app_types

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [16]:
# Binning rare 'APPLICATION_TYPE' values
rare_apps = app_types[app_types < 100].index
application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].apply(lambda x: 'Other' if x in rare_apps else x)

rare_apps

Index(['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object', name='APPLICATION_TYPE')

In [17]:
# Check the replacements of rare categories and verify changes
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [18]:
# Encoding categorical variables
application_df_encoded = pd.get_dummies(application_df)

In [23]:
# Features (X) and target (y)
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"]).values
y = application_df_encoded["IS_SUCCESSFUL"].values

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()

# Fit to the training data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
# Build the neural network model
nn_model = tf.keras.models.Sequential()

# Add layers to the model without regularization
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=X_train_scaled.shape[1]))  # First hidden layer
nn_model.add(tf.keras.layers.Dropout(0.5))  # Dropout layer with 50% rate

nn_model.add(tf.keras.layers.Dense(units=64, activation="relu"))  # Second hidden layer
nn_model.add(tf.keras.layers.Dropout(0.5))  # Dropout layer with 50% rate

nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))  # Third hidden layer
nn_model.add(tf.keras.layers.Dropout(0.5))  # Dropout layer with 50% rate

nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # Output layer

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, verbose=1)


Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5602 - loss: 0.7092 - val_accuracy: 0.7232 - val_loss: 0.6153
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6788 - loss: 0.6351 - val_accuracy: 0.7362 - val_loss: 0.5803
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7012 - loss: 0.6101 - val_accuracy: 0.7382 - val_loss: 0.5631
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7090 - loss: 0.5982 - val_accuracy: 0.7365 - val_loss: 0.5599
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7112 - loss: 0.6026 - val_accuracy: 0.7374 - val_loss: 0.5554
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7108 - loss: 0.5898 - val_accuracy: 0.7363 - val_loss: 0.5522
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7250 - loss: 0.5757 - val_accuracy: 0.7385 - val_loss: 0.5629
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7186 - loss: 0.5815 - val_accu

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=1)

# Print the accuracy and loss
print(f"Loss: {model_loss}")
print(f"Accuracy: {model_accuracy}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7272 - loss: 0.5541
Loss: 0.557885468006134
Accuracy: 0.7255101799964905
